<a href="https://colab.research.google.com/github/Josephrp/Auto-GPT-Forge/blob/master/Open_Interpreter%5BJARVIS%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to JARVIS.

## The core of JARVIS is powered by [Open Interpreter](https://openinterpreter.com/):

Please set your API keys to begin.

In [ ]:
from getpass import getpass

eleven_labs_api_key = getpass.getpass("Eleven Labs API Key: ") # https://elevenlabs.io/speech-synthesis
openai_api_key = getpass("OpenAI API Key: ") # https://platform.openai.com/account/api-keys

## Install (`Restart Runtime` after this)

In [ ]:
!pip install open-interpreter
!pip install git+https://github.com/openai/whisper.git -q
!pip install gradio -q
!pip install elevenlabs
!pip install tiktoken==0.5.0 --force -q

## Setup

Import dependencies, download and install [Whisper](https://github.com/openai/whisper), set [OpenAI API key](https://platform.openai.com/account/api-keys), and [Eleven Labs API key](https://elevenlabs.io/speech-synthesis)


#### Misc Imports

In [ ]:
import gradio as gr
import time

#### Download and load [Whisper](https://github.com/openai/whisper)

In [ ]:
import whisper
model = whisper.load_model("base")

#### Define transcription function


In [ ]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text

#### Set [OpenAI](https://openai.com) API Key

In [ ]:
import interpreter

interpreter.api_key = openai_api_key
interpreter.auto_run = True

#### Set [ElevenLabs](https://elevenlabs.io/) API Key

In [ ]:
from elevenlabs import generate, play, set_api_key

set_api_key(eleven_labs_api_key)

#### Define audio length

In [ ]:
import io
from pydub import AudioSegment

def get_audio_length(audio_bytes):
  # Create a BytesIO object from the byte array
  byte_io = io.BytesIO(audio_bytes)

  # Load the audio data with PyDub
  audio = AudioSegment.from_mp3(byte_io)

  # Get the length of the audio in milliseconds
  length_ms = len(audio)

  # Optionally convert to seconds
  length_s = length_ms / 1000.0

  return length_s

#### Define the speak function which handles audio generation

> Indented block



In [ ]:
def speak(text):
  speaking = True
  audio = generate(
      text=text,
      voice="Daniel"
  )
  play(audio, notebook=True)

  audio_length = get_audio_length(audio)
  time.sleep(audio_length)

# Run

In [ ]:
# @title JARVIS
# @markdown ### **Setup Instructions**
# @markdown 1. Run this cell, then scroll down to use the interface (don't click the link, and give the interface a minute to load).
# @markdown 2. Press the `Record from Microphone` button.
# @markdown 3. Allow access to your microphone, then speak your command.
# @markdown 4. Stop the recording, then press `Submit`.
# @markdown
# @markdown
# @markdown JARVIS will respond verbally + carry out your command.

last_sentence = ""

with gr.Blocks() as demo:

    chatbot = gr.Chatbot()
    audio_input = gr.inputs.Audio(source="microphone", type="filepath")
    btn = gr.Button("Submit")

    def transcribe(audio):
      audio = whisper.load_audio(audio)
      audio = whisper.pad_or_trim(audio)
      mel = whisper.log_mel_spectrogram(audio).to(model.device)
      _, probs = model.detect_language(mel)
      options = whisper.DecodingOptions()
      result = whisper.decode(model, mel, options)
      return result.text

    def add_user_message(audio, history):
        user_message = transcribe(audio)
        return history + [[user_message, None]]

    def bot(history):
        global last_sentence

        user_message = history[-1][0]
        history[-1][1] = ""
        active_block_type = ""
        language = ""
        for chunk in interpreter.chat(user_message, stream=True, display=False):

            # Message
            if "message" in chunk:
              if active_block_type != "message":
                active_block_type = "message"
              history[-1][1] += chunk["message"]

              last_sentence += chunk["message"]
              if any([punct in last_sentence for punct in ".?!\n"]):
                yield history
                speak(last_sentence)
                last_sentence = ""
              else:
                yield history

            # Code
            if "language" in chunk:
              language = chunk["language"]
            if "code" in chunk:
              if active_block_type != "code":
                active_block_type = "code"
                history[-1][1] += f"\n```{language}"
              history[-1][1] += chunk["code"]
              yield history

            # Output
            if "executing" in chunk:
              history[-1][1] += "\n```\n\n```text\n"
              yield history
            if "output" in chunk:
              if chunk["output"] != "KeyboardInterrupt":
                history[-1][1] += chunk["output"] + "\n"
                yield history
            if "active_line" in chunk and chunk["active_line"] == None:
              history[-1][1] = history[-1][1].strip()
              history[-1][1] += "\n```\n"
              yield history

        if last_sentence:
          speak(last_sentence)

    btn.click(add_user_message, [audio_input, chatbot], [chatbot]).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(debug=True)

In [ ]:
# @title Text-only JARVIS
# @markdown Run this cell for a ChatGPT-like interface.

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):

        user_message = history[-1][0]
        history[-1][1] = ""
        active_block_type = ""

        for chunk in interpreter.chat(user_message, stream=True, display=False):

            # Message
            if "message" in chunk:
              if active_block_type != "message":
                active_block_type = "message"
              history[-1][1] += chunk["message"]
              yield history

            # Code
            if "language" in chunk:
              language = chunk["language"]
            if "code" in chunk:
              if active_block_type != "code":
                active_block_type = "code"
                history[-1][1] += f"\n```{language}\n"
              history[-1][1] += chunk["code"]
              yield history

            # Output
            if "executing" in chunk:
              history[-1][1] += "\n```\n\n```text\n"
              yield history
            if "output" in chunk:
              if chunk["output"] != "KeyboardInterrupt":
                history[-1][1] += chunk["output"] + "\n"
                yield history
            if "end_of_execution" in chunk:
              history[-1][1] = history[-1][1].strip()
              history[-1][1] += "\n```\n"
              yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
demo.launch(debug=True)